In [67]:
import datetime as dt
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import  train_test_split
from sklearn.ensemble import RandomForestRegressor
import time
import datetime
pd.options.mode.chained_assignment = None

def show_split_time():
    split_time = time.time()
    elapsed = split_time - start_time
    print ("Split time:",str(datetime.timedelta(seconds=elapsed)))

In [68]:
pwd

'D:\\Semester 2 (Final)\\FYP (Final)\\Dataset\\Rossman'

In [69]:
import os
os.chdir ('D:\Semester 2 (Final)\FYP (Final)\Dataset\Rossman')

In [70]:
training_df = pd.read_csv("train.csv")
store_df = pd.read_csv("store.csv")
test_df = pd.read_csv("test.csv")

C:\Users\TP040804\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
def is_nan(val):
    return val != val

def less_than_ten(val):
    if int(val) < 10:
        return "0" + val
    else:
        return val

# 1.0 Train Dataset

### 1.1 Missing Value

In [72]:
training_df.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

# 2.0 Store Dataset

### 2.1 Missing Value

In [73]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [74]:
store_df[store_df['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


#### Competition distance is one of the feature which is related to CompetitionOpenSinceMonth and CompetitionOpenSinceYear due to if there is no competitor store open near Rossman store on a certain month or year then there will be no competition distance for that specific store based on that the null value of Competition distance is set to 0 if the value for CompetitionOpenSinceMonth and CompetitionOpenSinceYear is NaN or 0. 

In [75]:
# Fill NaN values in store_df for "CompetitionDistance" = 0 
store_df["CompetitionDistance"][is_nan(store_df["CompetitionDistance"])] = 0

#### There are 354 missing values for CompetitionOpenSinceMonth and CompetitionOpenSinceYear which cannot be deleted due to Promo 2 is applicable during that time as well as the Competition distance value is not zero that is why the null values are filling with their mean value. 

In [76]:
store_df['CompetitionOpenSinceMonth'] = store_df['CompetitionOpenSinceMonth'].fillna((store_df['CompetitionOpenSinceMonth'].mean()))

In [77]:
store_df['CompetitionOpenSinceYear'] = store_df['CompetitionOpenSinceYear'].fillna((store_df['CompetitionOpenSinceYear'].mean()))

#### There are 544 missing value for Promo2SinceWeek, Promo2SinceYear and PromoInterval. These three features are related to Promo2 due to this the missing value is replace with 0 when the Promo2 is 0.

In [78]:
print(store_df[store_df['Promo2SinceWeek'].isnull()])

      Store StoreType Assortment  CompetitionDistance  \
0         1         c          a               1270.0   
3         4         c          c                620.0   
4         5         a          a              29910.0   
5         6         a          a                310.0   
6         7         a          c              24000.0   
7         8         a          a               7520.0   
8         9         a          c               2030.0   
9        10         a          a               3160.0   
15       16         a          c               3270.0   
22       23         d          a               4060.0   
24       25         c          a                430.0   
25       26         d          a               2300.0   
28       29         d          c               2170.0   
30       31         d          c               9800.0   
32       33         a          c               1320.0   
33       34         c          a               2240.0   
36       37         c          

In [79]:
store_df.loc[store_df['Promo2'] == 0, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']] = 0

# 3.0 Test Dataset

In [80]:
test_df.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [81]:
test_df[test_df['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


#### There are 11 missing value for Open feature. Based on the research it shows that most of the shop is closed on Sunday due to this, the missing value is replacing with 1 if the DayofWeek is not Sunday. 

In [82]:
test_df["Open"][is_nan(test_df["Open"])] = (test_df["DayOfWeek"] != 7).astype(int)

# 4.0 Data Integration

### 4.1 Merging train and test dataset

In [83]:
train_store = pd.merge(training_df, store_df, how = 'left', on='Store')

In [84]:
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,31/7/2015,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,31/7/2015,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,31/7/2015,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,31/7/2015,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,31/7/2015,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [85]:
test_store = pd.merge(test_df, store_df, how = 'left', on='Store')

In [86]:
test_store.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,0.0,0.0,0
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,0.0,0.0,0
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,0.0,0.0,0


# 5.0 Preparing Data for Modelling

###### Converting Date to day, month and year

In [87]:
# Create "Year", "Month", "Day", "dayofyear", "week" columns
train_store['Year'] = pd.DatetimeIndex(train_store['Date']).year
train_store['Month'] = pd.DatetimeIndex(train_store['Date']).month
train_store['Day'] = pd.DatetimeIndex(train_store['Date']).day
train_store['DayOfYear'] = pd.DatetimeIndex(train_store['Date']).dayofyear

In [88]:
train_store['Week'] = pd.DatetimeIndex(train_store['Date']).week

In [89]:
test_store['Year'] = pd.DatetimeIndex(test_store['Date']).year
test_store['Month'] = pd.DatetimeIndex(test_store['Date']).month
test_store['Day'] = pd.DatetimeIndex(test_store['Date']).day
test_store['DayOfYear'] = pd.DatetimeIndex(test_store['Date']).dayofyear
test_store['Week'] = pd.DatetimeIndex(test_store['Date']).week

In [90]:
train_store.drop(["Date"], axis=1, inplace=True)

In [91]:
test_store.drop(["Date"], axis=1, inplace=True)

###### One_Hot Encoding

In [92]:
# "StateHoliday" has values "0" & 0
train_store["StateHoliday"].loc[train_store["StateHoliday"] == 0] = "0"
test_store["StateHoliday"].loc[test_store["StateHoliday"] == 0] = "0"

In [93]:
train_store = pd.get_dummies(train_store, columns=["DayOfWeek", "StateHoliday", "StoreType", "Assortment","SchoolHoliday","PromoInterval"])

In [94]:
test_store = pd.get_dummies(test_store, columns=["DayOfWeek", "StateHoliday", "StoreType", "Assortment", "SchoolHoliday", "PromoInterval"])

###### Remove Store Information in train_store which is not in test_store

In [95]:
train_store = train_store[train_store["Store"].isin(test_store.Store.unique())]

# Additional Features

###### Create new DataFrames for Average Customers per Store, per Store per Month & per Store per Year for "Customers" != 0

In [96]:
avg_cust = train_store.groupby(["Store"]).agg(
    {"Customers": {"AvgCustStore": lambda y: np.mean([x for x in y if x != 0])}})
avg_cust.columns = avg_cust.columns.get_level_values(1)
avg_cust.reset_index(inplace=True)

C:\Users\TP040804\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [97]:
avg_cust_month = train_store.groupby(["Store", "Month"]).agg(
    {"Customers": {"AvgCustStoreMonth": lambda y: np.mean([x for x in y if x != 0])}})
avg_cust_month.columns = avg_cust_month.columns.get_level_values(1)
avg_cust_month.reset_index(inplace=True)

C:\Users\TP040804\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [98]:
avg_cust_year = train_store.groupby(["Store", "Year"]).agg(
    {"Customers": {"AvgCustStoreYear": lambda y: np.mean([x for x in y if x != 0])}})
avg_cust_year.columns = avg_cust_year.columns.get_level_values(1)
avg_cust_year.reset_index(inplace=True)

C:\Users\TP040804\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


###### Computation for AvgSales, AvgCustomers, AvgSalesPerCustomer

In [99]:
totalSalesPerStore = train_store.groupby([train_store['Store']])['Sales'].sum()
totalCustomersPerStore = train_store.groupby([train_store['Store']])['Customers'].sum()
numberOfOpenStores = train_store.groupby([train_store['Store']])['Open'].count()

###### Average Sales for Open Days

In [100]:
AvgSalesPerStore = totalSalesPerStore / numberOfOpenStores
AvgCustomers = totalCustomersPerStore / numberOfOpenStores
AvgSalesPerCustomer = AvgSalesPerStore / AvgCustomers

###### Merge the newly created DataFrames with training & test dataset

In [101]:
train_store = pd.merge(train_store, AvgSalesPerCustomer.reset_index(name='AvgSalesPerCustomer'), how='left', on=['Store'])

In [102]:
test_store = pd.merge(test_store, AvgSalesPerCustomer.reset_index(name='AvgSalesPerCustomer'), how='left', on=['Store'])

In [103]:
train_store = pd.merge(train_store, AvgSalesPerStore.reset_index(name='AvgSalesPerStore'), how='left', on=['Store'])
#train_store = pd.merge(train_store, AvgCustomers.reset_index(name='AvgCustomers'), how='left', on=['Store'])

In [104]:
test_store = pd.merge(test_store, AvgSalesPerStore.reset_index(name='AvgSalesPerStore'), how='left', on=['Store'])
#test_store = pd.merge(test_store, AvgCustomers.reset_index(name='AvgCustomers'), how='left', on=['Store'])

In [105]:
train_store = pd.merge(train_store, avg_cust, on=["Store"])
train_store = pd.merge(train_store, avg_cust_month, on=["Store", "Month"])
train_store = pd.merge(train_store, avg_cust_year, on=["Store", "Year"])

In [106]:
test_store = pd.merge(test_store, avg_cust, on=["Store"])
test_store = pd.merge(test_store, avg_cust_month, on=["Store", "Month"])
test_store = pd.merge(test_store, avg_cust_year, on=["Store", "Year"])

###### Saving

In [107]:
train_store.to_csv("Working/Modelling_dataset/ModellingTrainData2.csv", sep=',', index=False)

In [108]:
test_store.to_csv("Working/Modelling_dataset/ModellingTestData2.csv", sep=',', index=False)

###### Remove all Closed Stores where Sales = 0 in train dataset

In [109]:
train_store = train_store[train_store["Open"] != 0]

# Log Tranformation

###### LogSales

In [110]:
LogSales = train_store['Sales']
train_store = train_store.drop(['Sales'], axis=1)

In [111]:
LogSales = np.log(LogSales + 1)

In [112]:
LogSales.head()

0    8.568646
1    8.521384
2    8.472823
3    8.519590
4    8.716536
Name: Sales, dtype: float64

In [113]:
train_store['Sales'] = LogSales 

In [114]:
train_store.head()

,Store,Customers,Open,Promo,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",AvgSalesPerCustomer,AvgSalesPerStore,AvgCustStore,AvgCustStoreMonth,AvgCustStoreYear,Sales
0,1,555,1,1,1270.0,9.0,2008.0,0,0.0,0.0,...,1,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.568646
1,1,546,1,1,1270.0,9.0,2008.0,0,0.0,0.0,...,1,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.521384
2,1,523,1,1,1270.0,9.0,2008.0,0,0.0,0.0,...,1,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.472823
3,1,560,1,1,1270.0,9.0,2008.0,0,0.0,0.0,...,1,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.519590
4,1,612,1,1,1270.0,9.0,2008.0,0,0.0,0.0,...,1,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.716536


###### Log_CompetitionDistance

###### Train dataset

In [115]:
LogCompetition = train_store['CompetitionDistance']
train_store = train_store.drop(['CompetitionDistance'], axis=1)

In [116]:
LogCompetition = np.log(LogCompetition + 1)

In [117]:
LogCompetition.head()

0    7.147559
1    7.147559
2    7.147559
3    7.147559
4    7.147559
Name: CompetitionDistance, dtype: float64

In [118]:
train_store['CompetitionDistance'] = LogCompetition 

In [119]:
train_store.head(10)

,Store,Customers,Open,Promo,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,...,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",AvgSalesPerCustomer,AvgSalesPerStore,AvgCustStore,AvgCustStoreMonth,AvgCustStoreYear,Sales,CompetitionDistance
0,1,555,1,1,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.568646,7.147559
1,1,546,1,1,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.521384,7.147559
2,1,523,1,1,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.472823,7.147559
3,1,560,1,1,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.519590,7.147559
4,1,612,1,1,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.716536,7.147559
6,1,500,1,0,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.381373,7.147559
7,1,459,1,0,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.217978,7.147559
8,1,503,1,0,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.234830,7.147559
9,1,463,1,0,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.150468,7.147559
10,1,469,1,0,9.0,2008.0,0,0.0,0.0,2015,...,0,0,0,8.437366,3945.704883,564.049936,547.68,531.171429,8.177235,7.147559


###### test dataset

In [120]:
LogCompetition2 = test_store['CompetitionDistance']
test_store = test_store.drop(['CompetitionDistance'], axis=1)

In [121]:
LogCompetition2 = np.log(LogCompetition2 + 1)

In [122]:
LogCompetition2.head()

0    7.147559
1    7.147559
2    7.147559
3    7.147559
4    7.147559
Name: CompetitionDistance, dtype: float64

In [123]:
test_store['CompetitionDistance'] = LogCompetition2

In [124]:
test_store.head()

,Id,Store,Open,Promo,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,...,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",AvgSalesPerCustomer,AvgSalesPerStore,AvgCustStore,AvgCustStoreMonth,AvgCustStoreYear,CompetitionDistance
0,1,1,1.0,1,9.0,2008.0,0,0.0,0.0,2015,...,1,0,0,0,8.437366,3945.704883,564.049936,522.309091,531.171429,7.147559
1,857,1,1.0,1,9.0,2008.0,0,0.0,0.0,2015,...,1,0,0,0,8.437366,3945.704883,564.049936,522.309091,531.171429,7.147559
2,1713,1,1.0,1,9.0,2008.0,0,0.0,0.0,2015,...,1,0,0,0,8.437366,3945.704883,564.049936,522.309091,531.171429,7.147559
3,2569,1,1.0,1,9.0,2008.0,0,0.0,0.0,2015,...,1,0,0,0,8.437366,3945.704883,564.049936,522.309091,531.171429,7.147559
4,3425,1,0.0,0,9.0,2008.0,0,0.0,0.0,2015,...,1,0,0,0,8.437366,3945.704883,564.049936,522.309091,531.171429,7.147559


###### Drop Customer and Open feature

In [125]:
train_store = train_store.drop(['Customers'], axis=1)

In [126]:
train_store = train_store.drop(['Open'], axis=1)

In [127]:
test_store = test_store.drop(['Open'], axis=1)

###### Saving for feature_Selection

In [128]:
train_store.to_csv("Working/FeatureSelection/ModellingTrainData2.csv", sep=',', index=False)

In [129]:
test_store.to_csv("Working/FeatureSelection/ModellingTestData2.csv", sep=',', index=False)